# Work with json and path

## Part 1

In [1]:
import os
import shutil
import glob
import fnmatch
import json
from nltk import FreqDist

### Folder structure: 
Data Processed<br>
&nbsp;&nbsp;&nbsp;&nbsp; terms<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;country<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;state<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;city<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;rating<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;price

#### create Data Processed folder

In [3]:
os.mkdir(qpath+'Data Processed')

#### loop through all files to get information

In [2]:
rpath = '/Users/Rui/Documents/INFO 7374/Assignment 2 - Requirements/Data/'  #Please change to cooresponding local folder while grading
qpath = '/Users/Rui/Documents/INFO 7374/Assignment2/'    
workpath = '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/'

In [4]:
allfiles = glob.glob(rpath+'*')
print(len(allfiles))

11728


#### try some methods, see how to get value names faster

In [5]:
term_list = []

for file in allfiles:
    term_list.append(json.loads(open(file).read())['term'])

In [6]:
term_list = FreqDist(term_list).most_common(100)

In [7]:
print(term_list)

[('restaurants', 6372), ('attractions', 4265), ('hotels', 1091)]


#### create a function to create folder

In [8]:
def create_folder(fdr_path, fdr_name):
    os.mkdir(os.path.join(fdr_path, fdr_name))

#### create term folders under Data Processed

In [9]:
for term in term_list:
    create_folder(workpath, term[0].title())

move files to coresponding term folder

In [10]:
for file in allfiles:
    ft = json.loads(open(file).read())['term']
    for term in term_list:
        if ft==term[0]:
            shutil.copy(file, workpath+term[0].title())

#### create country folders in each term folder

In [11]:
for term in term_list:
    wp = workpath + term[0].title() +'/'
    country_list = []
    for file in glob.glob(wp+'*'):
        country_list.append(json.loads(open(file).read())['location']['country'])
    country_list = FreqDist(country_list).most_common(100)
    for country in country_list:
        create_folder(wp, country[0])

#### move to country folders

In [12]:
for term in term_list:
    wp = workpath + term[0].title() +'/'
    for file in glob.glob(wp+'*'):
        if (os.path.isfile(file) == True):
            fc = json.loads(open(file).read())['location']['country']
            shutil.move(file, wp + fc + '/' + os.path.basename(file))

#### create state folders in each country 
do this by function is way better than loops...I can manually create a path instead of using 3 for loops : )

In [13]:
def create_state_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = os.path.join(workpath + t + '/' + c + '/')
    state_list = []
    for file in glob.glob(wp + '*'):
        if (os.path.isfile(file) == True):
            state_list.append(json.loads(open(file).read())['location']['state'])
    state_list = FreqDist(state_list).most_common(100)
    for state in state_list:
        create_folder(wp, state[0])

In [14]:
attraction_path = workpath+'Attractions/'
attraction_country_list = os.listdir(attraction_path)

for country in attraction_country_list:
    create_state_folders('attractions', country)

In [15]:
hotels_path =  workpath+'Hotels/'
hotels_country_list = os.listdir(hotels_path)
for country in hotels_country_list:
    create_state_folders('hotels', country)

In [16]:
restaurants_path =  workpath+'Restaurants/'
restaurants_country_list = os.listdir(restaurants_path)

for country in restaurants_country_list:
    create_state_folders('restaurants', country)

#### move to state folders
same thing...better to do with function

In [17]:
def move_to_state_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = os.path.join(workpath + t + '/' + c + '/')
    for file in glob.glob(wp+'*'):
        if (os.path.isfile(file) == True):
            fs = json.loads(open(file).read())['location']['state']
            shutil.move(file, wp + fs + '/' + os.path.basename(file))     

In [18]:
for country in attraction_country_list:
    move_to_state_folders('attractions', country)

In [19]:
for country in hotels_country_list:
    move_to_state_folders('hotels', country)

In [20]:
for country in restaurants_country_list:
    move_to_state_folders('restaurants', country)

In [21]:
glob.glob(workpath + 'Hotels/AU/*/')

['/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/ACT/',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/NSW/',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/QLD/',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/SA/',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/TAS/']

#### create city folders in each state

In [22]:
def create_city_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/' + '*/')
    for wp2 in wp:
        city_list = []
        for file in glob.glob(wp2 + '*'):
            if (os.path.isfile(file) == True):
                if json.loads(open(file).read())['location']['city'] != '':
                    city_list.append(json.loads(open(file).read())['location']['city'].title())
        city_list = FreqDist(city_list).most_common(100)
        for city in city_list:
            create_folder(wp2, city[0])

In [23]:
for country in attraction_country_list:
    create_city_folders('attractions', country)

In [24]:
for country in hotels_country_list:
    create_city_folders('hotels', country)

In [25]:
for country in restaurants_country_list:
    create_city_folders('restaurants', country)

#### move to city folders

In [26]:
def move_to_city_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/' + '*/')
    for wp2 in wp:  
        for file in glob.glob(wp2 +'*'):
            if (os.path.isfile(file) == True):                
                fci = json.loads(open(file).read())['location']['city'].title()
                shutil.move(file, wp2 + fci + '/' + os.path.basename(file))            

In [27]:
for country in attraction_country_list:
    move_to_city_folders('attractions', country)

In [28]:
for country in hotels_country_list:
    move_to_city_folders('hotels', country)

In [29]:
for country in restaurants_country_list:
    move_to_city_folders('restaurants', country)

#### create 0-1 1-2...4-5 5 rating folders accordingly for each city

In [30]:
def rate_func(f):
    rate_output = 0
    t_rate = json.loads(open(f).read())['rating']
    if 0 <= t_rate <= 1: 
        rate_output = 'Rating 0-1'
    elif 1 < t_rate <= 2:
        rate_output = 'Rating 1-2'
    elif 2 < t_rate <= 3:
        rate_output = 'Rating 2-3'
    elif 3 < t_rate <= 4:
        rate_output = 'Rating 3-4'   
    elif 4 < t_rate <=5:
        rate_output = 'Rating 4-5'
    return rate_output

In [31]:
glob.glob(workpath + 'Hotels/AU/ACT/*')[:4]

['/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/ACT/Acton',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/ACT/Barton',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/ACT/Belconnen',
 '/Users/Rui/Documents/INFO 7374/Assignment2/Data Processed/Hotels/AU/ACT/Bonython']

In [32]:
def create_rating_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/' + '*/')
    for wp2 in wp:                
        for wp3 in glob.glob(wp2 + '*'):
            rating_list = []
            for file in glob.glob(wp3 + '*/*'):   
                if os.path.isfile(file)==True:
                    rating = rate_func(file)                
                    rating_list.append(rating)
            rating_list = FreqDist(rating_list).most_common(5)
            for rt in rating_list:
                create_folder(wp3+'/', rt[0])

In [33]:
for country in attraction_country_list:
    create_rating_folders('attractions', country)

In [34]:
for country in hotels_country_list:
    create_rating_folders('hotels', country)

In [35]:
for country in restaurants_country_list:
    create_rating_folders('restaurants', country)

In [36]:
def move_to_rating_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/' + '*/*')
    for wp2 in wp:
        for file in glob.glob(wp2 + '/*'):
            if (os.path.isfile(file) == True):                
                filerating = rate_func(file)
                shutil.move(file, (wp2 + '/' + filerating + '/' + os.path.basename(file)))  

In [37]:
for country in attraction_country_list:
    move_to_rating_folders('attractions', country)

In [43]:
for country in hotels_country_list:
    move_to_rating_folders('hotels', country)

In [44]:
for country in restaurants_country_list:
    move_to_rating_folders('restaurants', country)

#### create \$ - \$\$\$\$\$ price folders accordingly for each rating

In [45]:
def create_price_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/*')
    for wp2 in wp:      
        for wp3 in glob.glob(wp2 + '/*'): 
            for wp4 in glob.glob(wp3 + '/*'):
                price_list = []                                    
                for file in glob.glob(wp4 + '/*'):
                    if os.path.isfile(file)==True:                          
                        if 'price' in json.loads(open(file).read()):
                            price = json.loads(open(file).read())['price']
                            price_list.append(price)
                price_list = FreqDist(price_list).most_common(100)
                for pr in price_list:
                    create_folder(wp4+'/', pr[0])

In [46]:
for country in attraction_country_list:
    create_price_folders('attractions', country)

In [47]:
for country in hotels_country_list:
    create_price_folders('hotels', country)

In [53]:
for country in restaurants_country_list:
    create_price_folders('restaurants', country)

In [49]:
def move_to_price_folders(t, c):
    t = t.title()
    c = c.upper()
    wp = glob.glob(workpath + t + '/' + c + '/' + '*/*')
    for wp2 in wp:
        for wp3 in glob.glob(wp2 + '/*'):
            for file in glob.glob(wp3 + '/*'):    
                if (os.path.isfile(file) == True):      
                    if 'price' in json.loads(open(file).read()):
                        price = json.loads(open(file).read())['price']
                        shutil.move(file, (wp3 + '/' + price + '/' + os.path.basename(file)))  

In [50]:
for country in attraction_country_list:
    move_to_price_folders('attractions', country)

In [51]:
for country in hotels_country_list:
    move_to_price_folders('hotels', country)

In [54]:
for country in restaurants_country_list:
    move_to_price_folders('restaurants', country)

#### clean up empty rating folders

In [58]:
for a in glob.glob(workpath+'*'):
    for b in glob.glob(a+'/*'):
        for c in glob.glob(b+'/*'):
            for d in glob.glob(c+'/*'):
                for e in glob.glob(d+'/*'):
                    if (os.path.isdir(e)):
                        if len(os.listdir(e+'/')) == 0:
                            os.rmdir(e)